# Document Retrieval Assignment

In [1]:
import graphlab

## 1. Compare top words according to word counts to TF-IDF

In [3]:
# Cargamos el dataset y creamos word_count y tf-idf

people = graphlab.SFrame('people_wiki.gl/')
people['word_count'] = graphlab.text_analytics.count_words(people['text'])

tfidf = graphlab.text_analytics.tf_idf(people['word_count'])

# Earlier versions of GraphLab Create returned an SFrame rather than a single SArray
# This notebook was created using Graphlab Create version 1.7.1
if graphlab.version <= '1.6.1':
    tfidf = tfidf['docs']

people['tfidf'] = tfidf

In [4]:
people.head(2)

Columns:
	URI	str
	name	str
	text	str
	word_count	dict
	tfidf	dict

Rows: 2

Data:
+-------------------------------+----------------+
|              URI              |      name      |
+-------------------------------+----------------+
| <http://dbpedia.org/resour... | Digby Morrell  |
| <http://dbpedia.org/resour... | Alfred J. Lewy |
+-------------------------------+----------------+
+-------------------------------+-------------------------------+
|              text             |           word_count          |
+-------------------------------+-------------------------------+
| digby morrell born 10 octo... | {'selection': 1, 'carltons... |
| alfred j lewy aka sandy le... | {'precise': 1, 'thomas': 1... |
+-------------------------------+-------------------------------+
+-------------------------------+
|             tfidf             |
+-------------------------------+
| {'selection': 3.8365785530... |
| {'precise': 6.443200606955... |
+-------------------------------+
[2 rows x 5 columns]

 In the notebook we covered in the module, we explored two document representations: word counts and TF-IDF. Now, take a particular famous person, 'Elton John'. What are the 3 words in his articles with highest word counts? What are the 3 words in his articles with highest TF-IDF? These results illustrate why TF-IDF is useful for finding important words.

In [5]:
elton = people[people['name'] == 'Elton John']

In [6]:
elton

Columns:
	URI	str
	name	str
	text	str
	word_count	dict
	tfidf	dict

Rows: Unknown

Data:
+-------------------------------+------------+-------------------------------+
|              URI              |    name    |              text             |
+-------------------------------+------------+-------------------------------+
| <http://dbpedia.org/resour... | Elton John | sir elton hercules john cb... |
+-------------------------------+------------+-------------------------------+
+-------------------------------+-------------------------------+
|           word_count          |             tfidf             |
+-------------------------------+-------------------------------+
| {'all': 1, 'least': 1, 'pr... | {'all': 1.6431112434912472... |
+-------------------------------+-------------------------------+
[? rows x 5 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.

In [7]:
elton[['tfidf']].stack('tfidf',new_column_name=['word','tfidf']).sort('tfidf',ascending=False)[:3]

word,tfidf
furnish,18.38947184
elton,17.48232027
billboard,17.3036809575


In [9]:
elton[['word_count']].stack('word_count',new_column_name=['word','word_count']).sort('word_count',ascending=False)[:3]

word,word_count
the,27
in,18
and,15


## 2. Measuring distance

 Elton John is a famous singer; let’s compute the distance between his article and those of two other famous singers. In this assignment, you will use the cosine distance, which one measure of similarity between vectors, similar to the one discussed in the lectures. You can compute this distance using the graphlab.distances.cosine function. What’s the cosine distance between the articles on ‘Elton John’ and ‘Victoria Beckham’? What’s the cosine distance between the articles on ‘Elton John’ and Paul McCartney’? Which one of the two is closest to Elton John? Does this result make sense to you?

In [10]:
victoria = people[people['name'] == 'Victoria Beckham']
paul = people[people['name'] == 'Paul McCartney']

In [11]:
graphlab.distances.cosine(elton['tfidf'][0],victoria['tfidf'][0])

0.9567006376655429

In [12]:
graphlab.distances.cosine(elton['tfidf'][0],paul['tfidf'][0])

0.8250310029221779

Está más cerca Paul McCartney.

## 3. Building nearest neighbors models with different input features and setting the distance metric

 In the sample notebook, we built a nearest neighbors model for retrieving articles using TF-IDF as features and using the default setting in the construction of the nearest neighbors model. Now, you will build two nearest neighbors models:
 * Using word counts as features
 * Using TF-IDF as features

In [16]:
# Creamos el modelo con word_count

knn_model_wc = graphlab.nearest_neighbors.create(people,
                                              features=['word_count'],
                                              label='name',
                                              distance='cosine')

Starting brute force nearest neighbors model training.

In [17]:
# Creamos el modelo con tf-idf

knn_model_tfidf = graphlab.nearest_neighbors.create(people,
                                              features=['tfidf'],
                                              label='name',
                                              distance='cosine')

Starting brute force nearest neighbors model training.

What’s the most similar article, other than itself, to the one on ‘Elton John’ using word count features?

In [18]:
knn_model_wc.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 48.345ms     |

| Done         |         | 100         | 1.00s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,2.22044604925e-16,1
0,Cliff Richard,0.16142415259,2
0,Sandro Petrone,0.16822542751,3
0,Rod Stewart,0.168327165587,4
0,Malachi O'Doherty,0.177315545979,5


What’s the most similar article, other than itself, to the one on ‘Elton John’ using TF-IDF features?

In [20]:
knn_model_tfidf.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 52.644ms     |

| 0            | 44545   | 75.4093     | 1.05s        |

| Done         |         | 100         | 1.40s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,-2.22044604925e-16,1
0,Rod Stewart,0.717219667893,2
0,George Michael,0.747600998969,3
0,Sting (musician),0.747671954431,4
0,Phil Collins,0.75119324879,5


What’s the most similar article, other than itself, to the one on ‘Victoria Beckham’ using word count features?

In [21]:
knn_model_wc.query(victoria)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 24.192ms     |

| 0            | 55809   | 94.4778     | 1.03s        |

| Done         |         | 100         | 1.10s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,-2.22044604925e-16,1
0,Mary Fitzgerald (artist),0.207307036115,2
0,Adrienne Corri,0.214509782788,3
0,Beverly Jane Fry,0.217466468741,4
0,Raman Mundair,0.217695474992,5


What’s the most similar article, other than itself, to the one on ‘Victoria Beckham’ using TF-IDF features?

In [22]:
knn_model_tfidf.query(victoria)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 62.864ms     |

| 0            | 26113   | 44.2061     | 1.06s        |

| 0            | 54273   | 91.8776     | 2.06s        |

| Done         |         | 100         | 2.20s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,1.11022302463e-16,1
0,David Beckham,0.548169610263,2
0,Stephen Dow Beckham,0.784986706828,3
0,Mel B,0.809585523409,4
0,Caroline Rush,0.819826422919,5
